In [1]:
import pandas as pd
import numpy as np



# from scipy.stats import spearmanr, rankdata
# from scipy.spatial.distance import pdist, squareform
import random
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import KFold

In [2]:
def load_data(path):

    files = os.listdir(f'{path}\\img\\data\\')
    image = np.empty((len(files), 50, 50, 1))
    sample = []
    id = []
    for i in range(len(files)):
        if files[i].split('.')[1] == 'txt':
            id.append(i)
            data = pd.read_csv(f'{path}\\img\\data\\{files[i]}', sep='\t', engine='c', na_values=['na', '-', ''], header=None, index_col=None)
            image[i, :, :, 0] = data.values
            sample.append(files[i].split('.txt')[0])
    image = image[id, :, :, :]

    return image

In [3]:
def top_n_miRNA(input_data, n):
    temp_df = input_data.copy()
    temp_df['sums'] = (temp_df.iloc[:,1:].sum(axis = 1))
    temp_df = temp_df.sort_values(by='sums', ascending=False)
    return temp_df.iloc[0:n,0:-1]


In [4]:
def balance_data(x_tn, y_lb):
    if(x_tn.shape[0]==y_lb.shape[0]):
        nonzero_index_r = y_lb.index[y_lb['amount']>0].to_list() #list of zero indices
        zero_index_r = y_lb.index[y_lb['amount']==0].to_list() #list of zero indices
        zero_index_r = random.choices(zero_index_r, k=len(nonzero_index_r))
        final_ind = nonzero_index_r + zero_index_r

        x_tn_temp = all_data[final_ind,:,:,:]
        y_lb_temp = y_lb.loc[final_ind]
        return x_tn_temp, y_lb_temp
    else:
        print('Dataset Shape did not match!!')

In [5]:
# def train_test_data(data_main, test_per = 0.2):
#     rand_test_index= np.random.randint(0, data_main.shape[0], np.int64(data_main.shape[0]*test_per))
#     test_data = data_main[rand_test_index]
#     train_data = np.delete(data_main, rand_test_index, axis = 0)

#     return train_data, test_data, rand_test_index


# train_data, test_data, test_indx = train_test_data(all_data, test_per=0.1)

In [6]:
# data_path = '..\\Results\\Table_To_Image_Conversion\\Test_1\\data'
data_path = "D:\\Research\\data\\processed_dataset" #lab

# tissue_type = 'liver'
# tissue_type = 'pancreas'
# tissue_type = 'kidney'
# tissue_type = 'heart'
# tissue_type = 'brain_myeloid'
tissue_type = 'marrow'


In [7]:
all_data = load_data(path = f'{data_path}\\{tissue_type}')
all_data.shape

(5355, 50, 50, 1)

In [8]:
all_data = all_data/255

In [9]:
y_target = pd.read_csv(f'{data_path}\\{tissue_type}\\miRNA_bulk.csv')
y_target = y_target.drop('Unnamed: 0',axis=1)

# y_target = y_target.iloc[:,1:].drop(y_target.index[(y_target.iloc[:,1:].sum(axis = 1) < 1000)]).T


y_target.shape

# y_target = y_target.iloc[:,1:].T
# y_target = y_target.values
# y_target

# y_target['sums'] = (y_target.iloc[:,1:].sum(axis = 1))
# y_target.sort_values(by='sums', ascending=False)
# y_target.iloc[0:50,0:-1]


(238, 5356)

In [10]:
top_miRNAs = top_n_miRNA(y_target, 1)

top_miRNAs = top_miRNAs.iloc[:,1:].T
top_miRNAs = top_miRNAs.rename(columns={top_miRNAs.columns[0]: "amount"})
sIdex = [i for i in range(top_miRNAs.shape[0])]
top_miRNAs = top_miRNAs.set_index([sIdex])
top_miRNAs


,amount
0,2
1,9
2,8
3,0
4,18
...,...
5350,2
5351,1
5352,3
5353,1


In [11]:
all_data, top_miRNAs = balance_data(all_data,top_miRNAs)
top_miRNAs_lbl = top_miRNAs.amount.values


In [12]:
top_miRNAs_lbl = top_miRNAs_lbl.reshape(-1, 1)

top_miRNAs_lbl = MinMaxScaler().fit_transform(top_miRNAs_lbl)


In [13]:
# total_sample = top_miRNAs.shape[0]
# ex_lvl = [0 for l in range(total_sample)]
# top_miRNAs['ex_lvl'] = ex_lvl
# top_miRNAs = top_miRNAs.reset_index()
# top_miRNAs


In [14]:
# for i in range(total_sample):
#     if(top_miRNAs['amount'][i]<10):
#         top_miRNAs['ex_lvl'][i] = 0
#     elif(top_miRNAs['amount'][i]<100):
#         top_miRNAs['ex_lvl'][i] = 1
#     else:
#         top_miRNAs['ex_lvl'][i] = 2


In [15]:
# top_miRNAs_lbl = top_miRNAs.ex_lvl.values
# encoded_array = np.zeros((top_miRNAs_lbl.size, top_miRNAs_lbl.max()+1), dtype=int)
# encoded_array[np.arange(top_miRNAs_lbl.size),top_miRNAs_lbl] = 1 
# top_miRNAs_lbl = encoded_array

In [16]:
# top_miRNAs_lbl


In [17]:
top_miRNAs_lbl

array([[0.01123596],
       [0.0505618 ],
       [0.04494382],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [18]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(top_miRNAs.iloc[:,1:])

# top_miRNAs = preprocessing.MinMaxScaler(top_miRNAs.iloc[:,1:].T)
# top_miRNAs
# top_miRNAs = top_miRNAs.iloc[:,1:].values.T

In [19]:
# top_miRNAs = top_miRNAs.T
# bins = [0, 99, 199, 299, 399, 499, 599, 699, 799, 899, 999]
# labels = ['J', 'I', 'H', 'G', 'F', 'E', 'D','C', 'B', 'A',]

def show_grade(smple):
    bins = [0, 1, 19, 29, 39, 49, 59, 69, 79, 89, 99, 999]
    labels = ['K','J', 'I', 'H', 'G', 'F', 'E', 'D','C', 'B', 'A',]
    smple['grade'] = pd.cut(x = smple['amount'], bins = bins, labels = labels, include_lowest = True)

In [20]:
show_grade(top_miRNAs)

In [21]:
# top_miRNAs.sort_values(by='amount')


In [22]:
# top_miRNAs['grade'].value_counts()
#

In [23]:
# quantiles  = top_miRNAs['grade'].value_counts().quantile([0.25, 0.5, 0.75])
# quantiles.plot(marker='o')
# plt.xlabel('Quantile')
# plt.ylabel('Value')
# plt.title('Quantiles')
# plt.show()

In [24]:
# for train_indices, test_indices in kfold.split(all_data):
#     y_train, y_val = top_miRNAs.values[train_indices], top_miRNAs.values[test_indices]


In [25]:

# def runCNN(train_x_input, train_y_input, val_x, val_y, epoch_num = 10, batch_num = 32):
# # Define the model
#     model = keras.Sequential()

#     # Add the layers to the model
#     model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#     model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#     model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
#     model.add(keras.layers.BatchNormalization())

#     # Flatten the output before passing it to the fully connected layers
#     model.add(keras.layers.Flatten())

#     # Add fully connected layers
#     model.add(keras.layers.Dense(128, activation='relu'))
#     model.add(keras.layers.Dense(64, activation='relu'))
#     model.add(keras.layers.Dense(train_y_input.shape[1], activation='softmax'))

#     # Compile the model
#     model.compile(optimizer='Adam',
#                 loss='mse',
#                 metrics=['accuracy'])

#     # Print the model summary
#     print(model.summary())

#     history = model.fit(train_x_input, train_y_input, epochs=epoch_num, batch_size=batch_num, validation_data = (val_x, val_y))

#     return model, history


In [26]:
def plot_history(his, tissue_type = ''):
    plt.plot(his.history['accuracy'])
    plt.plot(his.history['val_accuracy'])
    plt.title(f'Model Accuracy: {tissue_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(his.history['loss'])
    plt.plot(his.history['val_loss'])
    plt.title(f'Model Loss: {tissue_type}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


In [27]:
all_data_train, all_data_test, top_miRNAs_lbl_train, top_miRNAs_lbl_test = train_test_split(all_data, top_miRNAs_lbl, test_size=0.1, random_state=5)

print(f'X_train: {all_data_train.shape}')
print(f'X_test: {all_data_test.shape}')
print(f'y_train: {top_miRNAs_lbl_train.shape}')
print(f'y_test: {top_miRNAs_lbl_test.shape}')



X_train: (8125, 50, 50, 1)
X_test: (903, 50, 50, 1)
y_train: (8125, 1)
y_test: (903, 1)


In [28]:
model = keras.Sequential()

# Add the layers to the model
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(all_data.shape[1], all_data.shape[2], all_data.shape[3])))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())


# Flatten the output before passing it to the fully connected layers
model.add(keras.layers.Flatten())

# Add fully connected layers
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1))


# Define a custom learning rate
learning_rate = 0.0001
# optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer,
            loss='mean_squared_error',
            metrics=['accuracy'])

# Print the model summary
print(model.summary())




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        18464     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 32)       128       
 hNormalization)                                                 
                                                        

In [29]:
# model_pan, history_pan = runCNN(train_x_input=X_train, train_y_input=y_train, epoch_num=100, val_x=X_test,  val_y=y_test)

In [30]:
# plot_history(history_pan, tissue_type=tissue_type)

In [31]:
num_folds = 5
fold_scores = []
epoch_num = 50
batch_num = 8

kfold = KFold(n_splits=num_folds, shuffle=True)

# y_label = top_miRNAs.amount.values

for train_indices, test_indices in kfold.split(all_data_train):
    # print('Train: %s | test: %s' % (train_indices, test_indices))


    X_train, X_val = all_data_train[train_indices], all_data_train[test_indices]
    y_train, y_val = top_miRNAs_lbl_train[train_indices], top_miRNAs_lbl_train[test_indices]

    history = model.fit(X_train, y_train, epochs=epoch_num, batch_size=batch_num, validation_data = (X_val, y_val))

    _, score = model.evaluate(X_val, y_val)
    fold_scores.append(score)


Epoch 1/50
813/813 [==============================] - 5s 6ms/step - loss: 0.6661 - accuracy: 0.3926 - val_loss: 0.2120 - val_accuracy: 0.4782
Epoch 2/50
813/813 [==============================] - 4s 5ms/step - loss: 0.2965 - accuracy: 0.4292 - val_loss: 0.0772 - val_accuracy: 0.4738
Epoch 3/50
813/813 [==============================] - 4s 5ms/step - loss: 0.1962 - accuracy: 0.4468 - val_loss: 0.0582 - val_accuracy: 0.4738
Epoch 4/50
813/813 [==============================] - 4s 5ms/step - loss: 0.1442 - accuracy: 0.4609 - val_loss: 0.0466 - val_accuracy: 0.4757
Epoch 5/50
813/813 [==============================] - 4s 5ms/step - loss: 0.1059 - accuracy: 0.4712 - val_loss: 0.0289 - val_accuracy: 0.4782
Epoch 6/50
813/813 [==============================] - 4s 5ms/step - loss: 0.0900 - accuracy: 0.4806 - val_loss: 0.0258 - val_accuracy: 0.4763
Epoch 7/50
813/813 [==============================] - 4s 5ms/step - loss: 0.0748 - accuracy: 0.4851 - val_loss: 0.0214 - val_accuracy: 0.4825
Epoch 

In [32]:
np.mean(fold_scores)

0.4971076905727386

In [33]:
pred = model.predict(all_data_test)

29/29 [==============================] - 1s 12ms/step


In [34]:
from scipy.stats import spearmanr

rho, p = spearmanr(top_miRNAs_lbl_test, pred)

In [35]:
rho

0.09501351948816114